In [ ]:
# please run on colab; experimenting with yolov5xv6 with backbone frozen

In [ ]:
%%capture
!git clone https://github.com/ultralytics/yolov5  # clone
!cd yolov5
!pip install -r requirements.txt wandb

In [ ]:
from PIL import Image, ExifTags
from pycocotools.coco import COCO
from matplotlib.patches import Polygon, Rectangle
from matplotlib.collections import PatchCollection
import colorsys
import random
import pylab

import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
from tqdm import tqdm

import shutil
import os

import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils

In [ ]:
%%capture
!rm -rf /content/*
!mkdir data
!wget -O data/annotations.json https://raw.githubusercontent.com/pedropro/TACO/master/data/annotations.json
!wget https://raw.githubusercontent.com/pedropro/TACO/master/download.py
!python download.py

In [ ]:
# Read annotations
dataset_path = './data'
anns_file_path = dataset_path + '/' + 'annotations.json'

with open(anns_file_path, 'r') as f:
    dataset = json.loads(f.read())
imgs = dataset['images']

data_source = COCO(annotation_file=anns_file_path)

In [ ]:
!mkdir yoloTACO
!rm -rf ./yoloTACO/*
!mkdir yoloTACO/images
!mkdir yoloTACO/labels

dst = "./yoloTACO"
coco = COCO(anns_file_path)

for img in tqdm(imgs):

  # reorganize images
  img_id = img['id']
  img_location = './data/'+img['file_name']
  shutil.copy(img_location, os.path.join(dst+'/images/',str(img_id)+'.jpg'))

  # reorganize annotations
  annotation_ids = coco.getAnnIds(img_id)
  if len(annotation_ids) == 0:
    continue

  img_info = data_source.loadImgs(img_id)[0]    
  height = img_info['height']
  width = img_info['width']

  with open(dst+'/labels/'+str(img_id)+'.txt', mode='w') as fp:
    annotations = coco.loadAnns(annotation_ids)
    lines = ''
    for annotation in annotations:
      label = annotation['category_id']
      box = annotation['bbox'] 
      # some annotations have basically no width / height (extremely small), skip them
      if box[2] < 1 or box[3] < 1:
          continue
      # top_x,top_y,width,height ----> cen_x,cen_y,width,height
      # standardize to 0-1
      box[0] = round((box[0] + box[2] / 2) / width, 6)
      box[1] = round((box[1] + box[3] / 2) / height, 6)
      box[2] = round(box[2] / width, 6)
      box[3] = round(box[3] / height, 6)
      # line: label x_center y_center width height
      lines = lines + str(label)
      for i in box:
          lines += ' ' + str(i)
      lines += '\n'
    fp.writelines(lines)

In [ ]:
## train test split
'''
train: images/train
val: images/val
test: images/test'''
import random
id_list=[i for i in range(1500)]
random.shuffle(id_list)

train_ids = id_list[:1050]
val_ids = id_list[1050:1050+300]
test_ids = id_list[1050+300:]

def move_helper(ids, desti):
  for id in ids:
    img_name = os.path.join( './yoloTACO/images', str(id)+'.jpg' )
    lbl_name = os.path.join( './yoloTACO/labels', str(id)+'.txt' )
    print(img_name)
    if os.path.isfile(img_name):
        shutil.copy( img_name, 'yoloTACO/images/'+desti)
        shutil.copy( lbl_name, 'yoloTACO/labels/'+desti)
    else :
        print('file does not exist', img_name)

In [ ]:
!mkdir yoloTACO/images/train
!mkdir yoloTACO/images/val
!mkdir yoloTACO/images/test
!mkdir yoloTACO/labels/train
!mkdir yoloTACO/labels/val
!mkdir yoloTACO/labels/test
move_helper(test_ids,'test')
move_helper(train_ids,'train')
move_helper(val_ids,'val')

In [ ]:
%cd ./yolov5
!ls

In [ ]:
!nvidia-smi
!pip list -v | grep [Aa]lbumentations

In [ ]:
# Train YOLOv5s on COCO128 for 5 epochs
# run 'pip install wandb' to automatically track and visualize YOLOv5 🚀 runs in Weights & Biases
!python train.py --img 1024 --batch-size 8 --epochs 10 --data yoloTACO.yaml --weights yolov5x.pt --freeze 10 --multi-scale 


In [ ]:
!python val.py --data yoloTACO.yaml --weights runs/train/exp/weights/best.pt